In [ ]:
!pip install keras-tuner

In [ ]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D

from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from kerastuner.tuners import RandomSearch
from kerastuner.tuners import Hyperband
from kerastuner.tuners import BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters
import pickle

In [ ]:
%load_ext autotime
RANDOM_STATE = 42

In [ ]:
seed = 7
np.random.seed(seed)

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
def view_random_image(target_dir, target_class):
    target_folder = target_dir + target_class
    random_image = random.sample(os.listdir(target_folder), 1)
    img = mpimg.imread(target_folder+'/'+random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    plt.axis('off');
    #print(f"Image shape {img.shape}")
    return img

In [ ]:
import random
import matplotlib.image as mpimg
from pathlib import Path
import os.path
class_names=['buildings', 'forest', 'glacier', 'mountain' ,'sea' ,'street']
plt.figure(figsize=(20, 10))
for i in range(18):
    plt.subplot(3, 6, i+1)
    class_name = random.choice(class_names)
    img = view_random_image(target_dir='/home/abir/Downloads/greedyhpo/intel/archive/seg_train/seg_train/',target_class=class_name)

In [ ]:
train_dir = "/home/abir/Downloads/greedyhpo/intel/archive/seg_train/seg_train"
test_dir = "/home/abir/Downloads/greedyhpo/intel/archive/seg_test/seg_test"
val_dir ="/home/abir/Downloads/greedyhpo/intel/archive/seg_val/seg_test/"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data = ImageDataGenerator(rescale = 1./255)

training_data = train_data.flow_from_directory(train_dir,
                                              batch_size = 32,
                                              target_size = (32,32), 
                                              class_mode = 'categorical')

val_data = ImageDataGenerator(rescale = 1./255)

validation_data = val_data.flow_from_directory(val_dir,
                                            batch_size = 32,
                                            target_size = (32,32), 
                                            class_mode = 'categorical')


test_data = ImageDataGenerator(rescale = 1./255)

testing_data = test_data.flow_from_directory(test_dir,
                                            batch_size = 32,
                                            target_size = (32,32), 
                                            class_mode = 'categorical')

In [ ]:
LOG_DIR = f"{int(time.time())}"

In [ ]:
def build_model(hp):

    model = Sequential()
    model.add(Conv2D(hp.Choice('layer_size_1', values=[16,32,64,128,256,512],default=16), 3, activation='relu', padding = 'same', input_shape=(32, 32, 3),kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(Dropout(hp.Choice('dp_size_1', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0)))
    model.add(Conv2D(hp.Choice('layer_size_1', values=[16,32,64,128,256,512],default=16), 3, activation='relu', padding = 'same',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    #####################################Block 2#################################         
    model.add(Conv2D(hp.Choice('layer_size_2', values=[16,32,64,128,256,512],default=16), 3,padding ='same', activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(Dropout(hp.Choice('dp_size_2', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0)))
    model.add(Conv2D(hp.Choice('layer_size_2', values=[16,32,64,128,256,512],default=16), 3,padding ='same', activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    ######################################Block 3#################################            
    model.add(Conv2D(hp.Choice('layer_size_3', values=[16,32,64,128,256,512],default=16), 3, padding = 'same', activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(Dropout(hp.Choice('dp_size_3', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0)))
    model.add(Conv2D(hp.Choice('layer_size_3', values=[16,32,64,128,256,512],default=16), 3, padding = 'same', activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(Dropout(hp.Choice('dp_size_3', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0)))
    model.add(Conv2D(hp.Choice('layer_size_3', values=[16,32,64,128,256,512],default=16), 3, padding = 'same', activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    ######################################Block 4################################
    model.add(Conv2D(hp.Choice('layer_size_4', values=[16,32,64,128,256,512],default=16), 3, padding = 'same', activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(Dropout(hp.Choice('dp_size_4', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0)))
    model.add(Conv2D(hp.Choice('layer_size_4', values=[16,32,64,128,256,512],default=16), 3, padding = 'same', activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(Dropout(hp.Choice('dp_size_4', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0)))
    model.add(Conv2D(hp.Choice('layer_size_4', values=[16,32,64,128,256,512],default=16), 3, padding = 'same', activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    ######################################Block 5#################################
    model.add(Conv2D(hp.Choice('layer_size_5', values=[16,32,64,128,256,512],default=16), 3, padding = 'same', activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(Dropout(hp.Choice('dp_size_5', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0)))
    model.add(Conv2D(hp.Choice('layer_size_5', values=[16,32,64,128,256,512],default=16), 3, padding = 'same', activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(Dropout(hp.Choice('dp_size_5', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0)))
    model.add(Conv2D(hp.Choice('layer_size_5', values=[16,32,64,128,256,512],default=16), 3, padding = 'same', activation='relu',kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    ############################################################################
    model.add(Dropout(hp.Choice('dp_size_6', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0)))
    model.add(Flatten())
    model.add(Dense(hp.Choice('dense_size_1', values=[64,128,256,512],default=64),activation="relu",kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(Dense(hp.Choice('dense_size_2', values=[64,128,256,512],default=64),activation="relu",kernel_regularizer=regularizers.l2(hp.Choice('wd_size_1', values=[0.0,0.1, 0.001, 0.0001],default=0.0))))
    model.add(Dropout(hp.Choice('dp_size_6', values=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],default=0.0)))
    model.add(Dense(units=6, activation="softmax"))
 ##############################################################################           
    
    opt = Adam(lr=hp.Choice('learning_rate', values=[1e-1,1e-2, 1e-3, 1e-4, 1e-5],default=1e-2))
    
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                                                  optimizer=opt,
                                                  metrics=['accuracy'])
    return model

In [ ]:
with tf.device('/device:GPU:1'):
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=87,  # how many model variations to test?
        executions_per_trial=1,  # how many trials per variation? (same model could perform differently)
        directory=LOG_DIR,
        project_name='RS_Vgg16_Cifar10')

    tuner.search_space_summary()
    my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=3)]
    tuner.search(training_data,
                 verbose=1, # just slapping this here bc jupyter notebook. The console out was getting messy.
                 epochs=100,
                 batch_size=64,
                 callbacks=[my_callbacks],  # if you have callbacks like tensorboard, they go here.
                 validation_data=validation_data)

    tuner.results_summary()

    with open(f"tuner_{int(time.time())}.pkl", "wb") as f:
        pickle.dump(tuner, f)

In [ ]:
new_model = tuner.get_best_hyperparameters()[0]

In [ ]:
model = tuner.hypermodel.build(new_model)

In [ ]:
model.summary()

In [ ]:
with tf.device('/device:GPU:1'):
    hist=model.fit(training_data,
          batch_size=64,
          epochs=100,
          validation_data=validation_data,
          callbacks=[my_callbacks])

In [ ]:
_,test_acc = model.evaluate(testing_data[0][0],testing_data[0][1])